In [2]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoModel,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import random
import numpy as np

In [3]:
import pandas as pd
import re
import difflib
import json

In [4]:
ANNO_IDX = 435

In [5]:
# データの読み込み
full_data = pd.read_csv("/data/group1/z40436a/ME/URL_Citation_Classification_Intermediate/data/all_data.csv", encoding="utf-8", usecols=['id', 'url', 'passage-title', 'citation-info', 'citation-paragraph', 'citation-type', 'role','type', 'year'], index_col=0)
full_data['tsunokake_id'] = full_data.index
## 使用するのはRoleがMaterialとMethodであるもののみ
choice_data = full_data.query("role == 'Method' | role == 'Material'").reset_index().drop('id', axis=1)
display(choice_data.head())

,url,passage-title,citation-info,citation-paragraph,role,type,year,citation-type,tsunokake_id
0,https://github.com/mdelhoneux/uuparser-compos...,['4 Composition in a K&G Parser'],4 The code can be found at https://github.com/...,"Parser We use UUParser, a variant of the K&G t...",Method,Code,2019,Footnote,0
1,http://hdl.handle.net/11234/1-2364,['5 What Correlates with Difficulty?'],Milan Straka and Jana Strakov. 2017. Tokenizin...,Head-POS Entropy Dehouck and Denis (2018) prop...,Material,Knowledge,2019,Reference,1
2,http://sjmielke.com/papers/tokenize/,['D Data selection: Europarl'],31 http://sjmielke.com/papers/tokenize/,"Finally, it should be said that the text in Co...",Method,Tool,2019,Footnote,2
3,https://developer.twitter.com/en/docs.html,"['2 Problem Formulation', '2.2 Data']",NaN,• Negative examples: We have col-lected 1% of ...,Method,Tool,2019,Body,5
4,https://www.mturk.com/,['5 User study'],Amazon. 2005. MTurk. (https://www.mturk.com/).,To verify whether human evaluators are in agre...,Method,Tool,2019,Reference,6


In [6]:
sorted(choice_data['year'].unique())

[np.int64(2001),
 np.int64(2002),
 np.int64(2003),
 np.int64(2004),
 np.int64(2005),
 np.int64(2006),
 np.int64(2007),
 np.int64(2008),
 np.int64(2009),
 np.int64(2010),
 np.int64(2011),
 np.int64(2012),
 np.int64(2013),
 np.int64(2014),
 np.int64(2015),
 np.int64(2016),
 np.int64(2017),
 np.int64(2018),
 np.int64(2019),
 np.int64(2020),
 np.int64(2021)]

In [ ]:
# split train & test based on "year"
## full test data = 317
# test_df = choice_data[:ANNO_IDX-1].query("year==2021")
train_df = choice_data[:ANNO_IDX-1].query("year<2021")

test_df = choice_data.query("year==2021")
# train_df = choice_data.query("year<2021")

print("train_size:::", len(train_df))
print("test_size:::", len(test_df))

print("train_ids:::", train_df.index)
print("test_ids:::", test_df.index)

train_size::: 1655
test_size::: 317
train_ids::: Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971],
      dtype='int64', length=1655)
test_ids::: Index([  46,   48,   49,   50,   51,   52,   53,   65,   66,   68,
       ...
       1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1957, 1958],
      dtype='int64', length=317)
